# Customer Churn Prediction with Amazon SageMaker Autopilot

> Based on [AWS labs](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/autopilot/autopilot_customer_churn.ipynb)

## Introduction
Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (hence the name) or with human guidance, without code through SageMaker Studio, or using the AWS SDKs. This notebook, as a first glimpse, will use the AWS SDKs to simply create and deploy a machine learning model.

Losing customers is costly for any business. Identifying unhappy customers early on gives you a chance to offer them incentives to stay. This notebook describes using machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction. ML models rarely give perfect predictions though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

We use an example of churn that is familiar to all of us–leaving a mobile phone operator. Seems like I can always find fault with my provider du jour! And if my provider knows that I’m thinking of leaving, it can offer timely incentives–I can always use a phone upgrade or perhaps have a new feature activated–and I might just stick around. Incentives are often much more cost effective than losing and reacquiring a customer.

## Setup
Let's start by specifying:

+ The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
+ The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).


In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role
region = boto3.Session().region_name
session = sagemaker.Session()
# You can modify the following to use a bucket of your choosing
bucket = os.environ['DORA_BUCKET']
prefix = 'workspaces/didone/notebooks/churn/sagemaker/autopilot-churn'
role = "arn:aws:iam::229343956935:role/DoraSageMaker"
# This is the client we will use to interact with SageMaker AutoPilot
sm = boto3.Session().client(service_name='sagemaker',region_name=os.environ['AWS_REGION'])

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

## Data

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

In [3]:
%%sql --out churn_df
SELECT * 
  FROM CHURN

DataFrame[CEP: string, ESTADO: string, GENERO: string, DDD: string, CELULAR: string, IDADE: double, IDADE_CONTA: double, CREDITO: float, DEBITO: float, T_SERVICOS: float, T_EDUCACAO: float, T_RESTAURANTE: float, T_TRANSPORTE: float, T_LAZER: float, T_SUPERMERCADO: float, T_OUTROS: float, CHURN: string]

In [4]:
churn = churn_df.toPandas()
pd.set_option('display.max_columns', 500)
churn

,CEP,ESTADO,GENERO,DDD,CELULAR,IDADE,IDADE_CONTA,CREDITO,DEBITO,T_SERVICOS,T_EDUCACAO,T_RESTAURANTE,T_TRANSPORTE,T_LAZER,T_SUPERMERCADO,T_OUTROS,CHURN
0,55677,DF,male,47,50211417,53.775342,2.290411,700.840027,1544.959961,721.349976,413.160004,0.000000,0.000000,0.000000,0.000000,1111.290039,False.
1,43369,AM,male,46,16013498,74.328767,1.369863,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False.
2,83328,MG,female,13,35325396,27.331507,8.852055,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False.
3,46752,RO,male,15,95345287,64.942466,2.410959,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False.
4,82969,PE,male,37,28499799,67.339726,13.526027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,37529,AP,male,10,09231334,52.572603,11.726027,483.510010,0.000000,483.510010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False.
2873,10835,RO,female,44,02921384,72.263014,17.213699,1309.709961,864.960022,50.480000,634.369995,891.609985,302.429993,65.190002,230.589996,0.000000,True.
2874,89047,BA,male,95,86082525,48.956164,17.975342,0.000000,80.599998,0.000000,0.000000,0.000000,80.599998,0.000000,0.000000,0.000000,True.
2875,74183,ES,female,44,91899317,35.287671,17.353425,2043.489990,1874.719971,445.570007,871.780029,1047.119995,684.919983,0.000000,744.229980,124.589996,True.


### Reserve some data for calling inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.


In [5]:
train_data = churn.sample(frac=0.8,random_state=200)
test_data = churn.drop(train_data.index)
test_data_no_target = test_data.drop(columns=['CHURN'])

Now we'll upload these files to S3.

In [6]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(bucket=os.environ['DORA_BUCKET'], path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(bucket=os.environ['DORA_BUCKET'], path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://autonomous-sandbox/workspaces/didone/notebooks/churn/sagemaker/autopilot-churn/train/train_data.csv
Test data uploaded to: s3://autonomous-sandbox/workspaces/didone/notebooks/churn/sagemaker/autopilot-churn/test/test_data.csv


## Setting up the SageMaker Autopilot Job

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset.

The required inputs for invoking a Autopilot job are:

+ Amazon S3 location for input dataset and for all output artifacts
+ Name of the column of the dataset you want to predict (Churn? in this case)
+ An IAM role

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order by name, is expected to have a header row.

In [7]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'CHURN'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

You can also specify the type of problem you want to solve with your dataset (Regression, MulticlassClassification, BinaryClassification). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict).

Because the target attribute, Churn?, is binary, our model will be performing binary prediction, also known as binary classification. In this example we will let AutoPilot infer the type of problem for us.

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a Candidate because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job takes about four hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.


## Launching the SageMaker Autopilot Job

You can now launch the Autopilot job by calling the create_auto_ml_job API. We limit the number of candidates to 20 so that the job finishes in a few minutes.

In [8]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-churn-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig={'CompletionCriteria':
                                       {'MaxCandidates': 20}
                                      },
                      RoleArn=role)

AutoMLJobName: automl-churn-15-15-02-24


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:229343956935:automl-job/automl-churn-15-15-02-24',
 'ResponseMetadata': {'RequestId': '6857c5ec-71c2-4a66-aa37-bf9d1cdf51ca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6857c5ec-71c2-4a66-aa37-bf9d1cdf51ca',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Wed, 15 Apr 2020 15:02:24 GMT'},
  'RetryAttempts': 0}}

## Tracking SageMaker Autopilot job progress

SageMaker Autopilot job consists of the following high-level steps :

+ Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
+ Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
+ Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline).


In [9]:
print ('JobStatus - Secondary Status')
print('------------------------------')

describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - Starting
InProgress - Starting
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress 

In [10]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

{'AutoMLJobName': 'automl-churn-15-15-02-24',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:229343956935:automl-job/automl-churn-15-15-02-24',
 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://autonomous-sandbox/workspaces/didone/notebooks/churn/sagemaker/autopilot-churn/train'}},
   'TargetAttributeName': 'CHURN'}],
 'OutputDataConfig': {'S3OutputPath': 's3://autonomous-sandbox/workspaces/didone/notebooks/churn/sagemaker/autopilot-churn/output'},
 'RoleArn': 'arn:aws:iam::229343956935:role/DoraSageMaker',
 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 20}},
 'CreationTime': datetime.datetime(2020, 4, 15, 15, 2, 24, 610000, tzinfo=tzlocal()),
 'EndTime': datetime.datetime(2020, 4, 15, 15, 33, 55, 993000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 4, 15, 15, 33, 56, 27000, tzinfo=tzlocal()),
 'BestCandidate': {'CandidateName': 'tuning-job-1-bdf09913f8474abd98-010-f390565e',
  'FinalAutoMLJobObjectiveMet

## Results

Now use the describe_auto_ml_job API to look up the best candidate selected by the SageMaker Autopilot job.


In [11]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{'CandidateName': 'tuning-job-1-bdf09913f8474abd98-010-f390565e', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:binary_classification_accuracy', 'Value': 0.9891774654388428}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:229343956935:processing-job/db-1-c1d92238523b4c21b5b08da92009e3813c724676147242d1ae295b30fc', 'CandidateStepName': 'db-1-c1d92238523b4c21b5b08da92009e3813c724676147242d1ae295b30fc'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:229343956935:training-job/automl-chu-dpp2-1-9730936911a24dcca828dfcd3ed6750b7562fd4b275e4', 'CandidateStepName': 'automl-chu-dpp2-1-9730936911a24dcca828dfcd3ed6750b7562fd4b275e4'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:229343956935:transform-job/automl-chu-dpp2-rpb-1-5888847a8f4c4cba9c53ebaa74a0597f7b4216

Due to some randomness in the algorithms involved, different runs will provide slightly different results, but accuracy will be around or above $93\%$, which is a good result.

## Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [15]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
model_name = best_candidate_name + timestamp_suffix + "-model"
model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

epc_name = best_candidate_name + timestamp_suffix + "-epc"
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType': 'ml.m5.2xlarge',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': 'main'}])

ep_name = best_candidate_name + timestamp_suffix + "-ep"
create_endpoint_response = sm.create_endpoint(EndpointName=ep_name,
                                              EndpointConfigName=epc_name)

In [16]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=ep_name)

### Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request. But first, we'll need to setup serializers and deserializers for passing our test_data NumPy arrays to the model behind the endpoint.

In [17]:
from io import StringIO
from sagemaker.predictor import RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV


predictor = RealTimePredictor(
    endpoint=ep_name,
    sagemaker_session=session,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_CSV)

# Remove the target column from the test data
test_data_inference = test_data.drop('CHURN', axis=1)

# Obtain predictions from SageMaker endpoint
prediction = predictor.predict(test_data_inference.to_csv(sep=',', header=False, index=False)).decode('utf-8')

# Load prediction in pandas and compare to ground truth
prediction_df = pd.read_csv(StringIO(prediction), header=None)
accuracy = (test_data.reset_index()['CHURN'] == prediction_df[0]).sum() / len(test_data_inference)
print('Accuracy: {}'.format(accuracy))


Accuracy: 0.9895652173913043


## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. This operation deletes all the generated models and the auto-generated notebooks as well.

In [18]:
sm.delete_endpoint(EndpointName=ep_name)
sm.delete_endpoint_config(EndpointConfigName=epc_name)
sm.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '1e499246-df98-4187-b6b1-f4e5dda51d0b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1e499246-df98-4187-b6b1-f4e5dda51d0b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 15 Apr 2020 15:51:50 GMT'},
  'RetryAttempts': 0}}